In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#read Dataset
df = pd.read_csv('vgames2.csv', index_col=0)
num_df_before = df.shape[0]

#drop NAs
df = df.dropna().reset_index(drop = True)
num_df_after = df.shape[0]

#refine data
sales_col = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

df.loc[df.Year<=22,'Year'] += 2000
df.loc[df.Year<=99,'Year'] += 1900

for col in sales_col:
  df[col] = df[col].str.replace('M','')
  df[col] = [(
      ele.replace('K', '') if re.match('.*K', ele) else float(ele) * 1000
      ) for ele in df[col]]

#re-define data type
df[['Platform', 'Genre', 'Publisher']] = df[['Platform', 'Genre', 'Publisher']].astype('category')
df[sales_col] = df[sales_col].astype(int)
df = df.convert_dtypes()

print(f'The number of    RAW data: {num_df_before}\nThe number of not-NA data: {num_df_after}\nduration: {np.amin(df.Year)} ~ {np.amax(df.Year)}')
df.info()

The number of    RAW data: 16598
The number of not-NA data: 16241
duration: 1980 ~ 2020
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16241 entries, 0 to 16240
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         16241 non-null  string  
 1   Platform     16241 non-null  category
 2   Year         16241 non-null  Int64   
 3   Genre        16241 non-null  category
 4   Publisher    16241 non-null  category
 5   NA_Sales     16241 non-null  Int64   
 6   EU_Sales     16241 non-null  Int64   
 7   JP_Sales     16241 non-null  Int64   
 8   Other_Sales  16241 non-null  Int64   
dtypes: Int64(5), category(3), string(1)
memory usage: 926.5 KB


In [ ]:
df_wide = df[df.Year < 2017].groupby(['Year', 'Genre'])[sales_col].sum().fillna(0).reset_index()
df_wide['Total_Sales'] = np.mean(df_wide[sales_col].T)
df_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Year         444 non-null    int64   
 1   Genre        444 non-null    category
 2   NA_Sales     444 non-null    Int64   
 3   EU_Sales     444 non-null    Int64   
 4   JP_Sales     444 non-null    Int64   
 5   Other_Sales  444 non-null    Int64   
 6   Total_Sales  444 non-null    float64 
dtypes: Int64(4), category(1), float64(1), int64(1)
memory usage: 23.5 KB


In [ ]:
def make_genre_plot(df, genre):
  df_tmp = df[df.Genre == genre][['Year'] + sales_col].melt(id_vars= 'Year', var_name= 'Sales')
  sns.histplot(data= df_tmp, x = 'Year', y= 'value', hue= 'Sales', bins=12,kde= True)

In [ ]:
plt.figure(figsize= [16, 24])
for i, genre in enumerate(df.Genre.unique()):
  plt.subplot(4, 3, i+1)
  make_genre_plot(df_wide, genre)
  plt.title(genre)

plt.legend();


In [ ]:
#plt
sns.lineplot(df_wide, x='Year', y='Total_Sales', hue='Genre')

plt.legend();

In [24]:
df_tmp=pd.read_csv('vgames2.csv', index_col=0)
df_tmp[df_tmp.Genre.isna() == True]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
51,Ape Escape: On the Loose,PSP,2005.0,NaN,Sony Computer Entertainment,0.52,0.01,0.12,0.05
409,NBA Ballers,XB,2004.0,NaN,Midway Games,0.37,0.11,0,0.02
1329,NFL Street,XB,2004.0,NaN,Electronic Arts,0.49,0.15,0,0.02
1583,SingStar Singalong With Disney,PS2,2008.0,NaN,Sony Computer Entertainment,0,0.04,0,0.57
1972,Pokemon X/Pokemon Y,3DS,2013.0,NaN,Nintendo,5170K,4.05,4.34,0.79
2044,NASCAR Racing,PS,1996.0,NaN,Pioneer LDC,0.23,0.16,0,0.03
2498,Jurassic Park III: Island Attack,GBA,2001.0,NaN,Konami Digital Entertainment,0.13,0.05,0,0
3354,The Godfather: Dons Edition,PS3,2007.0,NaN,Electronic Arts,0.25,0.02,0,0.03
3364,Ski-Doo Snowmobile Challenge,X360,2009.0,NaN,Valcon Games,0.03,0,0,0
3471,Batman Beyond: Return of the Joker,N64,2000.0,NaN,Ubisoft,0.07,0.02,0,0
